# Summer Recap: Data Analysis with Pandas

* * * 

<div class="alert alert-success">  
    
### Learning Objectives 
    
* Review fundamental pandas operations for data manipulation and analysis.
* Apply data cleaning techniques to real-world social science datasets.
* Practice exploratory data analysis using descriptive statistics and basic visualizations.
* Demonstrate ability to filter, group, and aggregate data using pandas methods.
* Evaluate LLM-generated code for accuracy and best practices.
</div>

### Icons Used in This Notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
🥊 **Challenge**: Interactive exercise. We'll work through these in the workshop!<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
⚠️ **Warning:** Heads-up about tricky stuff or common mistakes.<br>
🤖 **AI Generated**: Code generated by an LLM that we'll test and debug.<br>

### Sections
1. [Data Loading and Initial Exploration](#section1)
2. [Data Cleaning and Basic Operations](#section2)
3. [Exploratory Data Analysis](#section3)
4. [Text Analysis Fundamentals](#section4)
5. [Working with LLM-Generated Code](#section5)

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

<a id='section1'></a>

# Data Loading and Initial Exploration

Today we'll work with data from Reddit's "Am I the Asshole?" (AITA) subreddit. This dataset contains posts where people describe situations and ask for community judgment about their behavior.

In [ ]:
# Load the dataset
df = pd.read_csv('../../data/aita_top_subs.csv')

# Display basic information about the dataset
print(f"Dataset shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())

In [ ]:
# Display the first few rows
df.head()

## 🥊 Challenge 1: Data Overview

Explore the dataset structure and provide a summary of what you find. Use pandas methods to:
1. Check the data types of each column
2. Look for missing values
3. Get basic descriptive statistics for numerical columns

In [ ]:
# STANDARD ANSWER - Challenge 1: Data Overview

# 1. Check the data types of each column
print("=== DATA TYPES ===")
print(df.dtypes)
print("\n" + "="*50 + "\n")

# 2. Look for missing values
print("=== MISSING VALUES ===")
# Method 1: Count of missing values per column
missing_counts = df.isnull().sum()
print("Missing value counts:")
print(missing_counts)

# Method 2: Percentage of missing values (more informative for large datasets)
missing_percentages = (df.isnull().sum() / len(df)) * 100
print("\nMissing value percentages:")
for col, pct in missing_percentages.items():
    if pct > 0:
        print(f"{col}: {pct:.2f}%")

print("\n" + "="*50 + "\n")

# 3. Get basic descriptive statistics for numerical columns
print("=== DESCRIPTIVE STATISTICS ===")
# Using describe() gives us count, mean, std, min, 25%, 50%, 75%, max
numerical_stats = df.describe()
print(numerical_stats)

# Additional useful info: data shape and memory usage
print(f"\n=== ADDITIONAL INFO ===")
print(f"Dataset shape: {df.shape} (rows, columns)")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Show unique values for categorical columns (first few)
print(f"\nUnique authors: {df['author'].nunique()}")
print(f"Sample authors: {df['author'].unique()[:5]}")

🔔 **Question**: What do you notice about the `selftext` column? What might this tell us about the data?

<a id='section2'></a>

# Data Cleaning and Basic Operations

Real-world data often requires cleaning before analysis. Let's examine our dataset for common issues.

In [ ]:
# Check for duplicate posts
print(f"Number of duplicate rows: {df.duplicated().sum()}")

# Look at the distribution of some key variables
print(f"\nScore statistics:")
print(df['score'].describe())

## 🥊 Challenge 2: Data Cleaning

Clean the dataset by:
1. Removing any posts where `selftext` is missing or empty
2. Creating a new column called `text_length` that contains the character count of `selftext`
3. Filter out posts that are shorter than 100 characters (likely low-quality posts)

In [ ]:
# STANDARD ANSWER - Challenge 2: Data Cleaning

print(f"Original dataset shape: {df.shape}")

# Step 1: Remove posts where selftext is missing or empty
# Check current state of selftext column
print(f"\nSelftext column analysis:")
print(f"Missing values: {df['selftext'].isnull().sum()}")
print(f"Empty strings: {(df['selftext'] == '').sum()}")

# Remove missing values first
df_clean = df.dropna(subset=['selftext']).copy()
print(f"After removing missing selftext: {df_clean.shape}")

# Remove empty strings (posts with no content)
df_clean = df_clean[df_clean['selftext'] != ''].copy()
print(f"After removing empty selftext: {df_clean.shape}")

# Step 2: Create text_length column
# Calculate character count for each selftext
df_clean['text_length'] = df_clean['selftext'].str.len()

print(f"\nText length statistics:")
print(df_clean['text_length'].describe())

# Step 3: Filter out posts shorter than 100 characters
# These are likely low-quality or incomplete posts
print(f"\nPosts with less than 100 characters: {(df_clean['text_length'] < 100).sum()}")

df_clean = df_clean[df_clean['text_length'] >= 100].copy()
print(f"After filtering short posts: {df_clean.shape}")

# Summary of cleaning process
print(f"\n=== CLEANING SUMMARY ===")
print(f"Original rows: {df.shape[0]}")
print(f"Final rows: {df_clean.shape[0]}")
print(f"Rows removed: {df.shape[0] - df_clean.shape[0]}")
print(f"Percentage retained: {(df_clean.shape[0] / df.shape[0]) * 100:.1f}%")

# Update our main dataframe for subsequent analysis
df = df_clean.copy()

print(f"\nFinal text length distribution:")
print(df['text_length'].describe())

💡 **Tip**: Use the `.str.len()` method to get string lengths in pandas. Remember that missing values might cause issues, so handle them first!

## Working with Dates

The `created` column contains Unix timestamps. Let's convert these to readable dates.

In [ ]:
# Convert Unix timestamp to datetime
df['created_date'] = pd.to_datetime(df['created'], unit='s')

# Extract useful date components
df['year'] = df['created_date'].dt.year
df['month'] = df['created_date'].dt.month
df['day_of_week'] = df['created_date'].dt.day_name()

print("Date range in dataset:")
print(f"From: {df['created_date'].min()}")
print(f"To: {df['created_date'].max()}")

<a id='section3'></a>

# Exploratory Data Analysis

Now let's explore patterns in the data using pandas grouping and aggregation functions.

## 🥊 Challenge 3: Score Analysis

Analyze post popularity by:
1. Finding the top 10 posts by score
2. Calculating the average score by year
3. Determining which day of the week gets the highest average scores

In [ ]:
# STANDARD ANSWER - Challenge 3: Score Analysis

# 1. Find the top 10 posts by score
print("=== TOP 10 POSTS BY SCORE ===")
top_posts = df.nlargest(10, 'score')[['title', 'score', 'num_comments', 'author']]
print(top_posts)

# Let's also show some details about the highest scoring post
print(f"\nHighest scoring post details:")
highest_post = df.loc[df['score'].idxmax()]
print(f"Score: {highest_post['score']}")
print(f"Title: {highest_post['title']}")
print(f"Author: {highest_post['author']}")
print(f"Text length: {highest_post['text_length']} characters")

print("\n" + "="*60 + "\n")

# 2. Calculate average score by year
print("=== AVERAGE SCORE BY YEAR ===")
# Group by year and calculate mean score
yearly_scores = df.groupby('year')['score'].agg(['mean', 'count', 'std']).round(2)
yearly_scores.columns = ['Average Score', 'Post Count', 'Standard Deviation']
print(yearly_scores)

# Add some context about the year with highest average
best_year = yearly_scores['Average Score'].idxmax()
print(f"\nBest performing year: {best_year} with average score of {yearly_scores.loc[best_year, 'Average Score']}")

print("\n" + "="*60 + "\n")

# 3. Determine which day of the week gets highest average scores
print("=== AVERAGE SCORE BY DAY OF WEEK ===")
# Group by day of week and calculate statistics
daily_scores = df.groupby('day_of_week')['score'].agg(['mean', 'count', 'median']).round(2)
daily_scores.columns = ['Average Score', 'Post Count', 'Median Score']

# Sort by average score (descending) to see best performing days
daily_scores_sorted = daily_scores.sort_values('Average Score', ascending=False)
print(daily_scores_sorted)

# Highlight the best day
best_day = daily_scores_sorted.index[0]
print(f"\nBest performing day: {best_day}")
print(f"Average score on {best_day}: {daily_scores_sorted.loc[best_day, 'Average Score']}")
print(f"This is based on {daily_scores_sorted.loc[best_day, 'Post Count']} posts")

# Additional insight: Weekend vs Weekday analysis
weekend_days = ['Saturday', 'Sunday']
weekday_avg = df[~df['day_of_week'].isin(weekend_days)]['score'].mean()
weekend_avg = df[df['day_of_week'].isin(weekend_days)]['score'].mean()

print(f"\n=== WEEKEND vs WEEKDAY COMPARISON ===")
print(f"Weekday average score: {weekday_avg:.2f}")
print(f"Weekend average score: {weekend_avg:.2f}")
print(f"Weekend posts score {((weekend_avg/weekday_avg - 1) * 100):+.1f}% compared to weekdays")

## Comment Engagement Analysis

Now, let's explore how users interact with posts by analyzing the volume and distribution of comments, which can highlight engagement patterns and community response.

In [ ]:
# Explore the relationship between text length and engagement
correlation = df[['text_length', 'score', 'num_comments']].corr()
print("Correlation matrix:")
print(correlation)

🔔 **Question**: What does the correlation tell us about the relationship between post length and engagement?

## 🥊 Challenge 4: Engagement Categories

Create engagement categories and analyze them:
1. Create a new column `engagement_level` with categories:
   - 'Low': score < 100
   - 'Medium': score 100-500
   - 'High': score 500-2000
   - 'Viral': score > 2000
2. Calculate the percentage of posts in each category
3. Find the average text length for each engagement level

In [ ]:
# STANDARD ANSWER - Challenge 4: Engagement Categories

# 1. Create engagement_level column with categories based on score
print("=== CREATING ENGAGEMENT CATEGORIES ===")

# First, let's see the score distribution to understand our categorization
print("Score distribution:")
print(df['score'].describe())

# Create engagement categories using pd.cut() or conditional logic
# Using conditional logic for clarity in teaching
def categorize_engagement(score):
    """
    Categorize engagement level based on post score
    """
    if score < 100:
        return 'Low'
    elif score < 500:
        return 'Medium'
    elif score < 2000:
        return 'High'
    else:
        return 'Viral'

# Apply the categorization
df['engagement_level'] = df['score'].apply(categorize_engagement)

# Alternative method using pd.cut (more efficient for large datasets):
# df['engagement_level'] = pd.cut(df['score'], 
#                                bins=[0, 100, 500, 2000, float('inf')], 
#                                labels=['Low', 'Medium', 'High', 'Viral'],
#                                include_lowest=True)

print("Engagement categories created successfully!")
print(f"Sample categories: {df['engagement_level'].head()}")

print("\n" + "="*60 + "\n")

# 2. Calculate percentage of posts in each category
print("=== ENGAGEMENT LEVEL DISTRIBUTION ===")
engagement_counts = df['engagement_level'].value_counts().sort_index()
engagement_percentages = df['engagement_level'].value_counts(normalize=True).sort_index() * 100

# Create a summary table
engagement_summary = pd.DataFrame({
    'Count': engagement_counts,
    'Percentage': engagement_percentages.round(2)
})

print(engagement_summary)

# Additional insights about the distribution
print(f"\nTotal posts analyzed: {len(df)}")
print(f"Most common engagement level: {engagement_counts.idxmax()}")
print(f"Viral posts (rare content): {engagement_counts['Viral']} posts ({engagement_percentages['Viral']:.2f}%)")

print("\n" + "="*60 + "\n")

# 3. Find average text length for each engagement level
print("=== TEXT LENGTH BY ENGAGEMENT LEVEL ===")
text_length_by_engagement = df.groupby('engagement_level')['text_length'].agg([
    'mean', 'median', 'std', 'count'
]).round(2)

# Rename columns for clarity
text_length_by_engagement.columns = ['Average Length', 'Median Length', 'Std Deviation', 'Post Count']

# Sort by engagement level (Low -> Medium -> High -> Viral)
desired_order = ['Low', 'Medium', 'High', 'Viral']
text_length_by_engagement = text_length_by_engagement.reindex(desired_order)

print(text_length_by_engagement)

# Additional analysis: correlation between text length and engagement
print(f"\n=== KEY INSIGHTS ===")
correlation = df['text_length'].corr(df['score'])
print(f"Correlation between text length and score: {correlation:.3f}")

# Find the "sweet spot" text length
viral_avg_length = text_length_by_engagement.loc['Viral', 'Average Length']
low_avg_length = text_length_by_engagement.loc['Low', 'Average Length']
print(f"Viral posts are on average {viral_avg_length - low_avg_length:.0f} characters longer than low-engagement posts")

# Show range analysis
print(f"\n=== TEXT LENGTH RANGES BY ENGAGEMENT ===")
for level in desired_order:
    level_data = df[df['engagement_level'] == level]['text_length']
    print(f"{level:6} engagement: {level_data.min():5.0f} - {level_data.max():6.0f} characters (range: {level_data.max() - level_data.min():6.0f})")

<a id='section4'></a>

# Text Analysis Fundamentals

Let's do some basic text analysis to understand the content patterns.

## 🥊 Challenge 5: Text Pattern Analysis

Analyze text patterns by:
1. Finding posts that contain the word "family" (case-insensitive)
2. Counting how many posts mention "wedding" or "marriage"
3. Creating a column indicating whether the post is about relationships (contains words like "boyfriend", "girlfriend", "husband", "wife")

In [ ]:
# STANDARD ANSWER - Challenge 5: Text Pattern Analysis

# 1. Find posts that contain the word "family" (case-insensitive)
print("=== POSTS MENTIONING 'FAMILY' ===")

# Create a boolean mask for posts containing "family"
family_mask = df['selftext'].str.contains('family', case=False, na=False)
family_posts = df[family_mask]

print(f"Total posts mentioning 'family': {family_posts.shape[0]}")
print(f"Percentage of all posts: {(family_posts.shape[0] / len(df)) * 100:.2f}%")

# Show some examples
print(f"\nSample family-related post titles:")
for i, title in enumerate(family_posts['title'].head(3), 1):
    print(f"{i}. {title}")

# Average engagement for family posts vs others
family_avg_score = family_posts['score'].mean()
non_family_avg_score = df[~family_mask]['score'].mean()
print(f"\nEngagement comparison:")
print(f"Family posts average score: {family_avg_score:.2f}")
print(f"Non-family posts average score: {non_family_avg_score:.2f}")
print(f"Difference: {family_avg_score - non_family_avg_score:+.2f}")

print("\n" + "="*60 + "\n")

# 2. Count posts that mention "wedding" or "marriage"
print("=== POSTS ABOUT WEDDINGS/MARRIAGE ===")

# Use regex pattern with | (OR operator) to search for either word
wedding_marriage_mask = df['selftext'].str.contains('wedding|marriage', case=False, na=False)
wedding_marriage_posts = df[wedding_marriage_mask]

print(f"Posts mentioning 'wedding' or 'marriage': {wedding_marriage_posts.shape[0]}")
print(f"Percentage of all posts: {(wedding_marriage_posts.shape[0] / len(df)) * 100:.2f}%")

# Break down by individual terms (some posts might mention both)
wedding_only = df['selftext'].str.contains('wedding', case=False, na=False).sum()
marriage_only = df['selftext'].str.contains('marriage', case=False, na=False).sum()
both_terms = df['selftext'].str.contains('wedding', case=False, na=False) & df['selftext'].str.contains('marriage', case=False, na=False)

print(f"\nBreakdown:")
print(f"Posts with 'wedding': {wedding_only}")
print(f"Posts with 'marriage': {marriage_only}")
print(f"Posts with both terms: {both_terms.sum()}")

# Show sample titles
print(f"\nSample wedding/marriage post titles:")
for i, title in enumerate(wedding_marriage_posts['title'].head(3), 1):
    print(f"{i}. {title}")

print("\n" + "="*60 + "\n")

# 3. Create relationship indicator column
print("=== RELATIONSHIP POSTS ANALYSIS ===")

# Define relationship terms to search for
relationship_terms = ['boyfriend', 'girlfriend', 'husband', 'wife', 'partner', 'fiancé', 'fiancée', 'spouse']

# Create a regex pattern that matches any of these terms
# Using word boundaries \\b to ensure we match complete words
relationship_pattern = '|'.join([f'\\b{term}\\b' for term in relationship_terms])

# Create the relationship indicator column
df['is_relationship_post'] = df['selftext'].str.contains(
    relationship_pattern, 
    case=False, 
    na=False, 
    regex=True
)

# Count relationship posts
relationship_posts = df[df['is_relationship_post']]
print(f"Posts about relationships: {relationship_posts.shape[0]}")
print(f"Percentage of all posts: {(relationship_posts.shape[0] / len(df)) * 100:.2f}%")

# Detailed breakdown by relationship type
print(f"\nBreakdown by relationship type:")
for term in relationship_terms:
    # Use word boundaries to avoid partial matches
    term_pattern = f'\\b{term}\\b'
    count = df['selftext'].str.contains(term_pattern, case=False, na=False, regex=True).sum()
    if count > 0:  # Only show terms that appear in the dataset
        print(f"{term.capitalize():12}: {count:4d} posts")

# Compare engagement: relationship posts vs non-relationship posts
rel_avg_score = relationship_posts['score'].mean()
non_rel_avg_score = df[~df['is_relationship_post']]['score'].mean()

print(f"\n=== RELATIONSHIP POST INSIGHTS ===")
print(f"Relationship posts average score: {rel_avg_score:.2f}")
print(f"Non-relationship posts average score: {non_rel_avg_score:.2f}")
print(f"Relationship posts score {((rel_avg_score/non_rel_avg_score - 1) * 100):+.1f}% compared to others")

# Show engagement distribution for relationship posts
print(f"\nRelationship posts by engagement level:")
rel_engagement_dist = relationship_posts['engagement_level'].value_counts().sort_index()
rel_engagement_pct = relationship_posts['engagement_level'].value_counts(normalize=True).sort_index() * 100

for level in ['Low', 'Medium', 'High', 'Viral']:
    if level in rel_engagement_dist.index:
        count = rel_engagement_dist[level]
        pct = rel_engagement_pct[level]
        print(f"{level:6}: {count:4d} posts ({pct:5.1f}%)")

# Sample relationship post titles
print(f"\nSample relationship post titles:")
for i, title in enumerate(relationship_posts['title'].head(3), 1):
    print(f"{i}. {title}")

💡 **Tip**: Use the `.str.contains()` method with pandas to search for text patterns. The `case=False` parameter makes the search case-insensitive.

## Author Analysis

Let's examine the authors. Do authors post multiple times? If so, who posts the most?

In [ ]:
# Analyze posting patterns by author
author_stats = df['author'].value_counts().head(10)
print("Top 10 most active authors:")
print(author_stats)

## 🥊 Challenge 6: Final Analysis

Combine multiple pandas operations to answer this question:
**"What are the characteristics of the most engaging posts about relationships?"**

Create an analysis that:
1. Filters for relationship-related posts
2. Groups them by engagement level
3. Calculates average text length, comment count, and any other relevant metrics
4. Presents a clear summary of your findings

In [ ]:
# STANDARD ANSWER - Challenge 6: Final Analysis
# "What are the characteristics of the most engaging posts about relationships?"

print("=== COMPREHENSIVE RELATIONSHIP POST ANALYSIS ===")
print("Research Question: What are the characteristics of the most engaging posts about relationships?")
print("="*80)

# Step 1: Filter for relationship-related posts
relationship_posts = df[df['is_relationship_post'] == True].copy()
print(f"Total relationship posts in dataset: {len(relationship_posts)}")
print(f"Percentage of all posts: {(len(relationship_posts) / len(df)) * 100:.2f}%")

print(f"\n{'='*80}")

# Step 2: Group by engagement level and analyze characteristics
print("=== RELATIONSHIP POSTS BY ENGAGEMENT LEVEL ===")

# Create comprehensive analysis by engagement level
engagement_analysis = relationship_posts.groupby('engagement_level').agg({
    'score': ['count', 'mean', 'median', 'std'],
    'num_comments': ['mean', 'median'],
    'text_length': ['mean', 'median'],
    'created_date': ['min', 'max']  # Date range
}).round(2)

# Flatten column names for easier reading
engagement_analysis.columns = [
    'Post_Count', 'Avg_Score', 'Median_Score', 'Score_StdDev',
    'Avg_Comments', 'Median_Comments',
    'Avg_Text_Length', 'Median_Text_Length',
    'Earliest_Post', 'Latest_Post'
]

# Sort by engagement level order
level_order = ['Low', 'Medium', 'High', 'Viral']
engagement_analysis = engagement_analysis.reindex(level_order)

print(engagement_analysis)

print(f"\n{'='*80}")

# Step 3: Deep dive into high-engagement relationship posts
print("=== CHARACTERISTICS OF HIGH-ENGAGEMENT RELATIONSHIP POSTS ===")

high_engagement = relationship_posts[relationship_posts['engagement_level'].isin(['High', 'Viral'])]
low_engagement = relationship_posts[relationship_posts['engagement_level'] == 'Low']

print(f"High-engagement relationship posts: {len(high_engagement)}")
print(f"Low-engagement relationship posts: {len(low_engagement)}")

# Compare characteristics
characteristics = {
    'Average Score': {
        'High-Engagement': high_engagement['score'].mean(),
        'Low-Engagement': low_engagement['score'].mean()
    },
    'Average Comments': {
        'High-Engagement': high_engagement['num_comments'].mean(),
        'Low-Engagement': low_engagement['num_comments'].mean()
    },
    'Average Text Length': {
        'High-Engagement': high_engagement['text_length'].mean(),
        'Low-Engagement': low_engagement['text_length'].mean()
    }
}

for metric, values in characteristics.items():
    high_val = values['High-Engagement']
    low_val = values['Low-Engagement']
    diff_pct = ((high_val / low_val) - 1) * 100 if low_val > 0 else 0
    print(f"\n{metric}:")
    print(f"  High-engagement: {high_val:.2f}")
    print(f"  Low-engagement:  {low_val:.2f}")
    print(f"  Difference: {diff_pct:+.1f}%")

print(f"\n{'='*80}")

# Step 4: Analyze posting patterns (day of week, time trends)
print("=== POSTING PATTERNS FOR HIGH-ENGAGEMENT RELATIONSHIP POSTS ===")

# Day of week analysis
day_analysis = high_engagement['day_of_week'].value_counts()
print("High-engagement relationship posts by day of week:")
print(day_analysis.sort_values(ascending=False))

# Most successful day
best_day = day_analysis.idxmax()
print(f"\nBest day for relationship posts: {best_day} ({day_analysis[best_day]} posts)")

# Year analysis (if multiple years in data)
if high_engagement['year'].nunique() > 1:
    year_analysis = high_engagement.groupby('year').agg({
        'score': 'mean',
        'num_comments': 'mean'
    }).round(2)
    print(f"\nPerformance by year:")
    print(year_analysis)

print(f"\n{'='*80}")

# Step 5: Content analysis of high-engagement relationship posts
print("=== CONTENT PATTERNS IN HIGH-ENGAGEMENT RELATIONSHIP POSTS ===")

# Analyze specific relationship terms in high-engagement posts
relationship_terms = ['boyfriend', 'girlfriend', 'husband', 'wife', 'partner', 'fiancé', 'fiancée']
print("Relationship type breakdown in high-engagement posts:")

term_analysis = {}
for term in relationship_terms:
    pattern = f'\\b{term}\\b'
    count = high_engagement['selftext'].str.contains(pattern, case=False, na=False, regex=True).sum()
    if count > 0:
        term_analysis[term] = count

# Sort by frequency
term_analysis = dict(sorted(term_analysis.items(), key=lambda x: x[1], reverse=True))
for term, count in term_analysis.items():
    pct = (count / len(high_engagement)) * 100
    print(f"  {term.capitalize():12}: {count:3d} posts ({pct:5.1f}%)")

print(f"\n{'='*80}")

# Step 6: Key findings summary
print("=== KEY FINDINGS SUMMARY ===")
print("\nCharacteristics of the most engaging relationship posts:")
print(f"1. LENGTH: High-engagement posts are {characteristics['Average Text Length']['High-Engagement'] - characteristics['Average Text Length']['Low-Engagement']:.0f} characters longer on average")
print(f"2. ENGAGEMENT: Generate {characteristics['Average Comments']['High-Engagement']:.0f} comments vs {characteristics['Average Comments']['Low-Engagement']:.0f} for low-engagement posts")
print(f"3. TIMING: Most successful on {best_day}s")

# Calculate content insights
high_engagement_viral_pct = (len(relationship_posts[relationship_posts['engagement_level'] == 'Viral']) / len(relationship_posts)) * 100
most_common_relationship = max(term_analysis.items(), key=lambda x: x[1])[0] if term_analysis else "partner"

print(f"4. CONTENT: '{most_common_relationship}' posts are most common in high-engagement category")
print(f"5. RARITY: Only {high_engagement_viral_pct:.2f}% of relationship posts achieve viral status")

# Final insight: correlation with family posts
family_relationship_overlap = relationship_posts['selftext'].str.contains('family', case=False, na=False).sum()
overlap_pct = (family_relationship_overlap / len(relationship_posts)) * 100
print(f"6. FAMILY CONNECTION: {overlap_pct:.1f}% of relationship posts also mention family dynamics")

print(f"\n{'='*80}")
print("CONCLUSION: Most engaging relationship posts tend to be longer, more detailed")
print("narratives that provide sufficient context for community judgment, posted on")
print(f"weekends, with '{most_common_relationship}' relationships being most commonly discussed.")

⚠️ **Warning**: When working with text data, always be mindful of missing values and different text encodings that might cause unexpected results.

<div class="alert alert-success">

## ❗ Key Points

* Pandas provides powerful tools for loading, cleaning, and exploring real-world datasets.
* Always start data analysis by understanding your dataset structure and checking for data quality issues.
* The `.groupby()` method is essential for aggregating data and finding patterns across categories.
* Text data requires special handling, including case-insensitive searches and pattern matching.
* Correlation analysis helps identify relationships between numerical variables.
* Creating categorical variables from continuous data enables different types of analysis.

</div>